In [1]:
# helper functions

# get http request headers
def getHeaders():
    headers = {
        "apikey":akey,
        "accept":"application/json"
    }
    return headers


# GET /property/address via zip code
def fetch(url, params):
    headers = getHeaders()
    res = requests.get(url, headers=headers, params=params).json()
    return res


# show load bar and wait for next request
def wait(t):    
    sys.stdout.write('.')
    sys.stdout.flush()
    time.sleep(t)
    

def f(x):
    if x >= 10:
        return 10
    elif x <= 0:
        return 1
       
    else:
        return x 
    

def g(x):
    if x >= 5:
        return 10
    elif x <= 0:
        return 1
    else:
        return 5 
    
    
def h(x):
    return x['pt_rankStars'] + x['pt_rankStatewidePercentage'] + x['pt_averageStandardScore'] + x['pt_rankMovement'] + x['pt_pupilTeacherRatio']


def i(x):
    return (x['high_score'] + x['middle_score'] + x['elementary_score']) / 15


def k(x):
    if x >= 100:
        return 10
    elif x == 100:
        return 7.5
    elif x >= 50 and  x < 100:
        return 5
    elif x > 0 and  x < 50:
        return 1
    else:
        return 0


def l(x):
    return (x['score_grocery_stores'] + x['score_avg_police'] + x['score_station'] + x['score_avg_museum'])

In [2]:
# Dependencies
import sys
import time
import json
import requests
import importlib
import numpy as np
import pandas as pd
from pprint import pprint
import matplotlib.pyplot as plt
from config import gkey, zkey, zws_id, akey

In [3]:
# get list of zipcodes
%run ./write_zips.ipynb
import zipcodes
importlib.reload(zipcodes)
required_zips = zipcodes.ziplist

Target city: atlanta
Search radius: 1


In [4]:
# get lat, lng coords from ziplist
url = "https://maps.googleapis.com/maps/api/geocode/json?"
response = []
latlngs = []

for z in required_zips:
    params = {
        "address":z,
        "key":gkey
    }

    res = requests.get(url, params=params).json()
    response.append({"response":res, "zipcode": z})

for r in response:
    try:
        zc = r['zipcode']
        lat = r['response']['results'][0]['geometry']['location']['lat']
        lng = r['response']['results'][0]['geometry']['location']['lng']
        
        latlngs.append({
            "lat":lat,
            "lng":lng,
            "zipcode":zc
        })

    except:
        continue

In [11]:
len(latlngs)

8

# Neighbourhood Analysis

In [12]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
ttypes = ['grocery_or_supermarket', 'police', 'transit_station', 'museum']

zipcodes = []
target_coordinates = []

cafes = []
museums = []
grocery_stores = []
police_stations = []
transit_stations = []

for l in latlngs:
    curzip = l['zipcode']
    curlatlng = f"{l['lat']},{l['lng']}"
    
    for i,t in enumerate(ttypes):
        curtype = t
        print(f"zipcode: {curzip}, latlng: {curlatlng}, target_type: {curtype}")
        
        params = {
            "target_search": curtype,
            "location": curlatlng,
            "radius": 1609, # TODO: pull this in for dynamic search radius
            "type": curtype,
            "key":gkey
        }

        response = requests.get(base_url, params=params).json()

        # populate list items
        for item in response["results"]:
            if(i == 0):
                grocery_stores.append({
                    "zipcode":curzip,
                    "loc":curlatlng,
                    "item":item["name"],
                    "type":curtype
                })
            elif(i == 1):
                police_stations.append({
                    "zipcode":curzip,
                    "loc":curlatlng,
                    "item":item["name"],
                    "type":curtype
                })
            elif(i == 2):
                transit_stations.append({
                    "zipcode":curzip,
                    "loc":curlatlng,
                    "item":item["name"],
                    "type":curtype
                })
            else:
                museums.append({
                    "zipcode":curzip,
                    "loc":curlatlng,
                    "item":item["name"],
                    "type":curtype
                })

zipcode: 30387, latlng: 28.048427,-82.7376544, target_type: grocery_or_supermarket
zipcode: 30387, latlng: 28.048427,-82.7376544, target_type: police
zipcode: 30387, latlng: 28.048427,-82.7376544, target_type: transit_station
zipcode: 30387, latlng: 28.048427,-82.7376544, target_type: museum
zipcode: 30312, latlng: 33.7440941,-84.377861, target_type: grocery_or_supermarket
zipcode: 30312, latlng: 33.7440941,-84.377861, target_type: police
zipcode: 30312, latlng: 33.7440941,-84.377861, target_type: transit_station
zipcode: 30312, latlng: 33.7440941,-84.377861, target_type: museum
zipcode: 30334, latlng: 33.7476898,-84.38804329999999, target_type: grocery_or_supermarket
zipcode: 30334, latlng: 33.7476898,-84.38804329999999, target_type: police
zipcode: 30334, latlng: 33.7476898,-84.38804329999999, target_type: transit_station
zipcode: 30334, latlng: 33.7476898,-84.38804329999999, target_type: museum
zipcode: 30302, latlng: 33.75,-84.39999999999999, target_type: grocery_or_supermarket
zip

In [66]:
# res_df = pd.DataFrame(response['results'])
# for g in res_df['geometry']:
#     pprint(g)

{'location': {'lat': 33.7563496, 'lng': -84.3734309},
 'viewport': {'northeast': {'lat': 33.7579126802915, 'lng': -84.37210646970848},
              'southwest': {'lat': 33.7552147197085, 'lng': -84.3748044302915}}}
{'location': {'lat': 33.7625707, 'lng': -84.39173269999999},
 'viewport': {'northeast': {'lat': 33.7637783802915, 'lng': -84.39049341970849},
              'southwest': {'lat': 33.7610804197085,
                            'lng': -84.39319138029151}}}
{'location': {'lat': 33.7489937, 'lng': -84.38811059999999},
 'viewport': {'northeast': {'lat': 33.7501413802915, 'lng': -84.38688101970848},
              'southwest': {'lat': 33.7474434197085, 'lng': -84.3895789802915}}}
{'location': {'lat': 33.75527, 'lng': -84.38308700000002},
 'viewport': {'northeast': {'lat': 33.7566179802915, 'lng': -84.3818554697085},
              'southwest': {'lat': 33.7539200197085,
                            'lng': -84.38455343029152}}}
{'location': {'lat': 33.7601908, 'lng': -84.395534},
 'viewp

In [48]:
columns = {
    "target_coordinates":latlngs,
    "grocery_stores":len(grocery_stores),
    "police":len(police_stations),
    "museum":len(museums),
    "transit_station":len(transit_stations)
}

nearby_df = pd.DataFrame(columns, columns=columns)
nearby_df

,target_coordinates,grocery_stores,police,museum,transit_station
0,"{'lat': 28.048427, 'lng': -82.7376544, 'zipcod...",16,113,111,160
1,"{'lat': 33.7440941, 'lng': -84.377861, 'zipcod...",16,113,111,160
2,"{'lat': 33.7476898, 'lng': -84.38804329999999,...",16,113,111,160
3,"{'lat': 33.75, 'lng': -84.39999999999999, 'zip...",16,113,111,160
4,"{'lat': 33.76, 'lng': -84.39, 'zipcode': '30343'}",16,113,111,160
5,"{'lat': 33.75, 'lng': -84.39, 'zipcode': '30353'}",16,113,111,160
6,"{'lat': 33.75, 'lng': -84.39, 'zipcode': '30392'}",16,113,111,160
7,"{'lat': 33.755711, 'lng': -84.3883717, 'zipcod...",16,113,111,160


In [ ]:
# # extract graocery stores, police stations, museums, transit stations and assign scores
# averagegrocery = placedata_df_pd["grocery_stores"].mean()
# avergaepolice = placedata_df_pd["police"].mean()
# avergaemuseum = placedata_df_pd["museum"].mean()
# avergaetransit_station = placedata_df_pd["transit_station"].mean()

# # Assign scores 
# placedata_df_pd["pg_avg_grocery_stores"] = (placedata_df_pd["grocery_stores"]/averagegrocery*100).fillna(0.0).astype(int)
# placedata_df_pd["pg_avg_police"] = (placedata_df_pd["police"]/avergaepolice*100 ).fillna(0.0).astype(int)
# placedata_df_pd["pg_avg_museum"] = (placedata_df_pd["museum"]/avergaemuseum*100 ).fillna(0.0).astype(int)
# placedata_df_pd["pg_avg_transit_station"] = placedata_df_pd["transit_station"]/avergaetransit_station*100

In [ ]:
# placedata_df_pd["score_grocery_stores"] = placedata_df_pd.apply(lambda row: k(row['pg_avg_grocery_stores']), axis=1)
# placedata_df_pd["score_avg_police"] = placedata_df_pd.apply(lambda row: k(row['pg_avg_police']), axis=1)
# placedata_df_pd["score_station"] = placedata_df_pd.apply(lambda row: k(row['pg_avg_transit_station']), axis=1)
# placedata_df_pd["score_avg_museum"] = placedata_df_pd.apply(lambda row: k(row['pg_avg_museum']), axis=1)
# placedata_df_pd["total_score"] = placedata_df_pd.apply(lambda row: l(row), axis=1)
# placedata_df_pd

# School Analysis

In [ ]:
# Inititalize arrays for all the various parameters returned from response of call that is needed
postal = []
schoolLevel = []
rankStars = []
rankStatewidePercentage = []
averageStandardScore = []
rankMovement = []
pupilTeacherRatio = []

In [ ]:
school_base_url="https://api.schooldigger.com/v1.1/schools?"

for zip in required_zips:
    params = {
        "appID":'cdcbd9a2',
        "appKey":'f21463a388bc041e25afa5dc32fb5014',
        "st":'GA',
        "zip":zip
    }

    # assemble url and make API request
    response = requests.get(school_base_url, params=params).json()
    response_df = pd.DataFrame(response)
    
    numberofresults = len(response_df)
    for i in range(numberofresults):
        schoolLevel.append(response_df["schoolList"][i].get("schoolLevel",np.nan))
        rankMovement.append(response_df["schoolList"][i].get("rankMovement",np.nan))
        postal.append(response_df["schoolList"][i]["address"].get("zip",np.nan))            

        try:
            rankStars.append(response_df["schoolList"][i]["rankHistory"][0].get("rankStars",np.nan))
            rankStatewidePercentage.append(response_df["schoolList"][i]["rankHistory"][0].get("rankStatewidePercentage",np.nan))
            averageStandardScore.append(response_df["schoolList"][i]["rankHistory"][0].get("averageStandardScore",np.nan))

        except (KeyError, IndexError, TypeError):
            rankStars.append(np.nan)
            rankStatewidePercentage.append(np.nan)
            averageStandardScore.append(np.nan)
                
        try:    
            pupilTeacherRatio.append(response_df["schoolList"][i]["schoolYearlyDetails"][0].get("pupilTeacherRatio",np.nan))
    
        except (KeyError, IndexError, TypeError):
            pupilTeacherRatio.append(np.nan)

        time.sleep(30)

In [ ]:
# create a data frame from all schools
school_dict = {
        "postal":postal,
        "schoolLevel":schoolLevel,
        "rankStars":rankStars,
        "rankStatewidePercentage":rankStatewidePercentage,
        "averageStandardScore":averageStandardScore,
        "rankMovement":rankMovement,
        "pupilTeacherRatio":pupilTeacherRatio
}

school_dict_pd = pd.DataFrame(school_dict)
school_dict_pd.replace(np.nan, 0, inplace=True)
school_dict_pd

In [ ]:
# seperate middle, elementary and high and assign scores
school_elementary_pd = school_dict_pd[school_dict_pd['schoolLevel'] == 'Elementary']
school_elementary_pd = school_elementary_pd.copy()
averagerankStars = school_elementary_pd["rankStars"].mean()

avergaerankStatewidePercentage = school_elementary_pd["rankStatewidePercentage"].mean()
avergaeaverageStandardScore = school_elementary_pd["averageStandardScore"].mean()
avergaerankMovement = school_elementary_pd["rankMovement"].mean()
avergaepupilTeacherRatio = school_elementary_pd["pupilTeacherRatio"].mean()

# Assign scores
school_elementary_pd["pg_avg_rankStars"] = (school_elementary_pd["rankStars"]/averagerankStars *10).fillna(0.0).astype(int)
school_elementary_pd["pg_avg_rankStatewidePercentage"] = (school_elementary_pd["rankStatewidePercentage"]/avergaerankStatewidePercentage *10).fillna(0.0).astype(int)
school_elementary_pd["pg_avg_averageStandardScore"] = (school_elementary_pd["averageStandardScore"]/avergaeaverageStandardScore *10).fillna(0.0).astype(int)
school_elementary_pd["pg_avg_rankMovement"] = school_elementary_pd["rankMovement"]/avergaerankMovement *10
school_elementary_pd["pg_avg_pupilTeacherRatio"] = (school_elementary_pd["pupilTeacherRatio"]/avergaepupilTeacherRatio *10).fillna(0.0).astype(int)

school_elementary_pd['pt_rankStars'] = school_elementary_pd.apply(lambda row: f(row['pg_avg_rankStars']), axis=1)
school_elementary_pd['pt_rankStatewidePercentage'] = school_elementary_pd.apply(lambda row: f(row['pg_avg_rankStatewidePercentage']), axis=1)
school_elementary_pd['pt_averageStandardScore'] = school_elementary_pd.apply(lambda row: f(row['pg_avg_averageStandardScore']), axis=1)
school_elementary_pd['pt_rankMovement'] = school_elementary_pd.apply(lambda row: g(row['rankMovement']), axis=1)
school_elementary_pd['pt_pupilTeacherRatio'] = school_elementary_pd.apply(lambda row: f(row['pg_avg_pupilTeacherRatio']), axis=1)
school_elementary_pd['total_points'] = school_elementary_pd.apply(lambda row: h(row),axis=1)
school_elementary_points = school_elementary_pd[['postal','total_points']]
elementary_groupby = school_elementary_points.groupby('postal').mean()

# elementary_groupby
school_elementary_pd

In [ ]:
# seperate middle, middle and high and assign scores
school_middle_pd = school_dict_pd[school_dict_pd['schoolLevel']=='Middle']
school_middle_pd = school_middle_pd.copy()
averagerankStars = school_middle_pd["rankStars"].mean()
avergaerankStatewidePercentage = school_middle_pd["rankStatewidePercentage"].mean()
avergaeaverageStandardScore = school_middle_pd["averageStandardScore"].mean()
avergaerankMovement = school_middle_pd["rankMovement"].mean()
avergaepupilTeacherRatio = school_middle_pd["pupilTeacherRatio"].mean()

# Assign scores
school_middle_pd["pg_avg_rankStars"] = (school_middle_pd["rankStars"]/averagerankStars *10).fillna(0.0).astype(int)
school_middle_pd["pg_avg_rankStatewidePercentage"] = (school_middle_pd["rankStatewidePercentage"]/avergaerankStatewidePercentage *10).fillna(0.0).astype(int)
school_middle_pd["pg_avg_averageStandardScore"] = (school_middle_pd["averageStandardScore"]/avergaeaverageStandardScore *10).fillna(0.0).astype(int)
school_middle_pd["pg_avg_rankMovement"] = school_middle_pd["rankMovement"]/avergaerankMovement *10
school_middle_pd["pg_avg_pupilTeacherRatio"] = (school_middle_pd["pupilTeacherRatio"]/avergaepupilTeacherRatio *10).fillna(0.0).astype(int)

school_middle_pd['pt_rankStars'] = school_middle_pd.apply(lambda row: f(row['pg_avg_rankStars']), axis=1)
school_middle_pd['pt_rankStatewidePercentage'] = school_middle_pd.apply(lambda row: f(row['pg_avg_rankStatewidePercentage']), axis=1)
school_middle_pd['pt_averageStandardScore'] = school_middle_pd.apply(lambda row: f(row['pg_avg_averageStandardScore']), axis=1)
school_middle_pd['pt_rankMovement'] = school_middle_pd.apply(lambda row: g(row['rankMovement']), axis=1)
school_middle_pd['pt_pupilTeacherRatio'] = school_middle_pd.apply(lambda row: f(row['pg_avg_pupilTeacherRatio']), axis=1)
school_middle_pd['total_points'] = school_middle_pd.apply(lambda row: h(row),axis=1)

school_middle_points=school_middle_pd[['postal','total_points']]
middle_groupby=school_middle_points.groupby('postal').mean()
middle_groupby

In [ ]:
# seperate high, high and high and assign scores
school_high_pd = school_dict_pd[school_dict_pd['schoolLevel']=='High']
school_high_pd = school_high_pd.copy()
averagerankStars = school_high_pd["rankStars"].mean()
avergaerankStatewidePercentage = school_high_pd["rankStatewidePercentage"].mean()
avergaeaverageStandardScore = school_high_pd["averageStandardScore"].mean()
avergaerankMovement = school_high_pd["rankMovement"].mean()
avergaepupilTeacherRatio = school_high_pd["pupilTeacherRatio"].mean()

# Assign scores
school_high_pd["pg_avg_rankStars"] = (school_high_pd["rankStars"]/averagerankStars *10).fillna(0.0).astype(int)
school_high_pd["pg_avg_rankStatewidePercentage"] = (school_high_pd["rankStatewidePercentage"]/avergaerankStatewidePercentage *10).fillna(0.0).astype(int)
school_high_pd["pg_avg_averageStandardScore"] = (school_high_pd["averageStandardScore"]/avergaeaverageStandardScore *10).fillna(0.0).astype(int)
school_high_pd["pg_avg_rankMovement"] = school_high_pd["rankMovement"]/avergaerankMovement *10
school_high_pd["pg_avg_pupilTeacherRatio"] = (school_high_pd["pupilTeacherRatio"]/avergaepupilTeacherRatio *10).fillna(0.0).astype(int)

school_high_pd['pt_rankStars'] = school_high_pd.apply(lambda row: f(row['pg_avg_rankStars']), axis=1)
school_high_pd['pt_rankStatewidePercentage'] = school_high_pd.apply(lambda row: f(row['pg_avg_rankStatewidePercentage']), axis=1)
school_high_pd['pt_averageStandardScore'] = school_high_pd.apply(lambda row: f(row['pg_avg_averageStandardScore']), axis=1)
school_high_pd['pt_rankMovement'] = school_high_pd.apply(lambda row: g(row['rankMovement']), axis=1)
school_high_pd['pt_pupilTeacherRatio'] = school_high_pd.apply(lambda row: f(row['pg_avg_pupilTeacherRatio']), axis=1)
school_high_pd['total_points'] = school_high_pd.apply(lambda row: h(row),axis=1)

school_high_points = school_high_pd[['postal','total_points']]
high_groupby = school_high_points.groupby('postal').mean()
high_groupby

In [ ]:
zip_final_school_score = []

combined_df = pd.merge(high_groupby, middle_groupby, how='outer', on='postal')

zip_final_school_score = pd.merge(combined_df, elementary_groupby, how='outer', on='postal')
zip_final_school_score.replace(np.nan, 0, inplace=True)
zip_final_school_score.rename(columns={'total_points_x': 'high_score','total_points_y': 'middle_score', 'total_points': 'elementary_score'}, inplace=True)
zip_final_school_score['final_score'] = zip_final_school_score.apply(lambda row: i(row), axis=1)
zip_final_school_score

# Price Analysis

In [ ]:
# get properties for each zip code (10 queries/min)
url = "https://search.onboard-apis.com/propertyapi/v1.0.0/property/address?"
property_results = []
prop_types = ['RESIDENTIAL (NEC)', #, 'CONDOMINIUM', 
              'APARTMENT'] #, 'CABIN', 'DUPLEX', 
              #'MANUFACTURED HOME', 'MOBILE HOME',
              #'TRIPLEX', 'TOWNHOUSE/ROWHOUSE']

sys.stdout.write('working..')

for z in latlngs:
    for pt in prop_types:
        params = {
            "postalcode": z['zipcode'],
            "propertytype": pt,
            "pagesize": "100"
        }

        res = fetch(url, params)

        property_results.append(res)
        wait(8)

In [ ]:
# get attomIds from property results
properties = []
attomIds = []

for p in property_results:
    for q in p['property']:
        properties.append(q)

props_df = pd.DataFrame(properties)
attomId_series = props_df['identifier']

for a in attomId_series:
    attomIds.append(a['attomId'])

In [ ]:
# get home values for each property (10 queries/min)
url = "https://search.onboard-apis.com/propertyapi/v1.0.0/property/basicprofile?"
sys.stdout.write("working..")
hv_results = []

for a in attomIds:
    params = {
        "attomId": a,
        "page":"1",
        "pagesize":"100"
    }

    idres = fetch(url, params)
    hv_results.append(idres)    
    wait(10)

In [ ]:
prop_res = pd.DataFrame(hv_results)
props = prop_res['property']
zcmv = []

for i in range(len(props)):
    zcmv.append({
        "zipcode":props[i][0]['address']['postal1'], 
        "avg_mkt_val":int(props[i][0]['assessment']['market']['mktTtlValue'])
    })

zips_mktvals_df = pd.DataFrame(zcmv).set_index('zipcode')
mkt_avg = zips_mktvals_df.groupby('zipcode').sum()/len(props)

mkt_avg_df = pd.DataFrame(mkt_avg)
mkt_avg_df = mkt_avg_df.reset_index()
mkt_avg_df.style.format({'avg_mkt_val':'${:,.2f}'})